In [1]:
# import necessary libraries
import numpy as np
import pandas as pd 
import seaborn as sns
import logging
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from scipy import stats
from scipy.stats import skew, norm
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pickle
from urllib.parse import urlparse
import dvc.api
import mlflow
import mlflow.sklearn
import mlflow.xgboost
import mlflow.keras
# ignore warnings
import warnings
warnings.filterwarnings(action="ignore")

In [2]:
#Create and configure logger
logging.basicConfig(filename="newfile.log",
                    format='%(asctime)s %(message)s',
                    filemode='w')

In [3]:
#Creating an object
logger=logging.getLogger()

In [4]:
path = ['data/store.csv', 'data/train.csv', 'data/test.csv']
version = ['store_v1', 'train_v1', 'test_v1']

In [5]:
    repo = '/home/michael/Pharmaceutical-Sales-prediction-across-multiple-stores'
    data_url = dvc.api.get_url(
        path = path[0],
        repo = repo,
        rev=version[0]
        )
    data_url2 = dvc.api.get_url(
        path = path[1],
        repo = repo,
        rev=version[1]
        )
    data_url3 = dvc.api.get_url(
        path = path[2],
        repo = repo,
        rev=version[2]
        )

    df_store = pd.read_csv(data_url)
    df_train = pd.read_csv(data_url2)
    df_test = pd.read_csv(data_url3)

In [6]:
# merge the train/test sets with the stores set
merged_train = pd.merge(left = df_train, right = df_store, how = 'inner', left_on = 'Store', right_on = 'Store')
merged_test = pd.merge(left = df_test, right = df_store, how = 'inner', left_on = 'Store', right_on = 'Store')
print(merged_train.shape)
print(merged_test.shape)

(1017209, 18)
(41088, 17)


In [7]:
deep_train = merged_train.copy()
deep_test = merged_test.copy()

In [8]:
def preprocess_data(train, test):
    
    # '''preprocessing'''
    global train_features, test_features, train_target, categorical, numerical

    # train and target features
    train_features = train.drop(['Sales', 'Customers'], axis = 1) #drop the target feature + customers (~ will not be used for prediction)
    train_target  = train[['Sales']]
    test_features = test.drop(['Id'], axis = 1) #drop id, it's required only during submission

    #feature generation + transformations
    train_features['Date'] = pd.to_datetime(train_features.Date)
    train_features['Month'] = train_features.Date.dt.month.to_list()
    train_features['Year'] = train_features.Date.dt.year.to_list()
    train_features['Day'] = train_features.Date.dt.day.to_list()
    train_features['WeekOfYear'] = train_features.Date.dt.weekofyear.to_list()
    train_features['DayOfWeek'] = train_features.Date.dt.dayofweek.to_list()
    train_features['weekday'] = 1        # Initialize the column with default value of 1
    train_features.loc[train_features['DayOfWeek'] == 5, 'weekday'] = 0
    train_features.loc[train_features['DayOfWeek'] == 6, 'weekday'] = 0
    train_features = train_features.drop(['Store'], axis = 1)

    test_features['Date'] = pd.to_datetime(test_features.Date)
    test_features['Month'] = test_features.Date.dt.month.to_list()
    test_features['Year'] = test_features.Date.dt.year.to_list()
    test_features['Day'] = test_features.Date.dt.day.to_list()
    test_features['WeekOfYear'] = test_features.Date.dt.weekofyear.to_list()
    test_features['DayOfWeek'] = test_features.Date.dt.dayofweek.to_list()
    test_features['weekday'] = 1        # Initialize the column with default value of 1
    test_features.loc[test_features['DayOfWeek'] == 5, 'weekday'] = 0
    test_features.loc[test_features['DayOfWeek'] == 6, 'weekday'] = 0
    test_features = test_features.drop(['Store'], axis = 1)


    # numerical and categorical columns (train set)
    categorical = []
    numerical = []
    timestamp = []

    for col in train_features.columns:
        if train_features[col].dtype == object:
            categorical.append(col)
        elif train_features[col].dtype in ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']:
            numerical.append(col)
        else:
            timestamp.append(col)

    # Keep selected columns only
    my_cols = categorical + numerical + timestamp
    train_features = train_features[my_cols].copy()
    test_features = test_features[my_cols].copy()
    features = pd.concat([train_features, test_features]) #merge the features columns for uniform preprocessing

    # change dtypes for uniformity in preprocessing
    features.CompetitionOpenSinceMonth = features.CompetitionOpenSinceMonth.astype('Int64') 
    features.CompetitionOpenSinceYear = features.CompetitionOpenSinceYear.astype('Int64')
    features.Promo2SinceWeek = features.Promo2SinceWeek.astype('Int64') 
    features.Promo2SinceYear = features.Promo2SinceYear.astype('Int64')
    features["StateHoliday"].loc[features["StateHoliday"] == 0] = "0"



    # ''' actual preprocessing: the mighty pipeline '''
    # numeric
    for col in ['CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2SinceWeek', 'Promo2SinceYear']:
        features[col] = features[col].fillna((int(features[col].mean()))) 
    features.PromoInterval = features.PromoInterval.fillna(features.PromoInterval.mode()[0])
    features.Open = features.Open.fillna(features.Open.mode()[0])
    features = pd.get_dummies(features, columns=['StoreType', 'Assortment', 'PromoInterval', 'StateHoliday'])
    
    scaler = RobustScaler()
    c = ['DayOfWeek', 'Open', 'Promo', 'SchoolHoliday', 'CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear',
    'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'WeekOfYear', 'Month', 'Year', 'Day', 'WeekOfYear', 'weekday']
    features[numerical] = scaler.fit_transform(features[numerical].values)


    return features

In [9]:
features = preprocess_data(merged_train, merged_test)
features = features.drop(['Date'], axis = 1)
features.head()

,DayOfWeek,Open,Promo,SchoolHoliday,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,...,Assortment_a,Assortment_b,Assortment_c,"PromoInterval_Feb,May,Aug,Nov","PromoInterval_Jan,Apr,Jul,Oct","PromoInterval_Mar,Jun,Sept,Dec",StateHoliday_0,StateHoliday_a,StateHoliday_b,StateHoliday_c
0,0.25,0.0,1.0,1.0,-0.172078,0.666667,0.0,-1.0,0.0,0.0,...,1,0,0,0,1,0,1,0,0,0
1,0.00,0.0,1.0,1.0,-0.172078,0.666667,0.0,-1.0,0.0,0.0,...,1,0,0,0,1,0,1,0,0,0
2,-0.25,0.0,1.0,1.0,-0.172078,0.666667,0.0,-1.0,0.0,0.0,...,1,0,0,0,1,0,1,0,0,0
3,-0.50,0.0,1.0,1.0,-0.172078,0.666667,0.0,-1.0,0.0,0.0,...,1,0,0,0,1,0,1,0,0,0
4,-0.75,0.0,1.0,1.0,-0.172078,0.666667,0.0,-1.0,0.0,0.0,...,1,0,0,0,1,0,1,0,0,0


In [10]:
# Transfomer data to time series problem
def reconstruct_sets(features):
    global x_train, x_val, y_train, y_val
    # global train_set
    # original train and test sets
    x_train = features.iloc[:len(train_features), :]
    x_test = features.iloc[len(train_features):, :]
    y_train = train_target
    # train_set = pd.concat([x_train, y_train], axis=1)

    # updated train and validation sets
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = .20, random_state = 0)


    return x_train, x_val, y_train, y_val, x_test

x_train, x_val, y_train, y_val, x_test = reconstruct_sets(features)
# log transformation on target variable
y_train = np.log1p(y_train['Sales'])
y_val = np.log1p(y_val['Sales'])
x_train.shape, x_val.shape, y_train.shape, y_val.shape, x_test.shape

((813767, 29), (203442, 29), (813767,), (203442,), (41088, 29))

In [13]:
with mlflow.start_run():
    alpha = 11
    learning_rate = 0.09
    xgb = XGBRegressor(learning_rate = learning_rate, alpha = alpha, random_state = 5)
    xgb.fit(x_train, y_train)
    xgb_pred = xgb.predict(x_val)
    print("Mean absolute error for xgb on validation data =", mean_absolute_error(y_val, xgb_pred))
    print("Mean squared error for xgb on validation data =", mean_squared_error(y_val, xgb_pred))
    print("Mean R2 score for xgb on validation data =", r2_score(y_val, xgb_pred))

    mlflow.log_param("learning_rate", learning_rate)
    mlflow.log_param("alpha", alpha)
    mlflow.log_metric("mse", mean_squared_error(y_val, xgb_pred))
    mlflow.log_metric("r2", r2_score(y_val, xgb_pred))
    mlflow.log_metric("mae", mean_absolute_error(y_val, xgb_pred))

    tracking_url_type_store = urlparse(mlflow.get_tracking_uri(),).scheme

    # Model registry does not work with file store
    if tracking_url_type_store != "file":

        mlflow.xgboost.log_model(xgb, "xgb model", registered_model_name="XGBRegressor")
    else:
        mlflow.xgboost.log_model(xgb, "xgb model")

Mean absolute error for xgb on validation data = 0.18399730159320643
Mean squared error for xgb on validation data = 0.06874030684376421
Mean R2 score for xgb on validation data = 0.9937185103595642


In [16]:
with mlflow.start_run():
    normalize = True
    copy_X = False
    regressor = LinearRegression(fit_intercept=True, normalize=normalize, copy_X=copy_X, n_jobs=None)
    regressor.fit(x_train, y_train)
    regressor_pred = regressor.predict(x_val)
    
    print("Elasticnet model (normalize=%f, l1_copy_x=%f):" % (normalize, copy_X))
    print("Mean squared error for SLR on validation data =", mean_squared_error(y_val, regressor_pred))
    print("Mean absolute error for SLR on validation data =", mean_absolute_error(y_val, regressor_pred))
    print("Mean R2 score for xgb on validation data =", r2_score(y_val, regressor_pred))

    mlflow.log_param("normalize", normalize)
    mlflow.log_param("copy_X", copy_X)
    mlflow.log_metric("mse", mean_squared_error(y_val, regressor_pred))
    mlflow.log_metric("r2", r2_score(y_val, regressor_pred))
    mlflow.log_metric("mae", mean_absolute_error(y_val, regressor_pred))

    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

    # Model registry does not work with file store
    if tracking_url_type_store != "file":

        mlflow.sklearn.log_model(regressor, "model", registered_model_name="LinearRegression")
    else:
        mlflow.sklearn.log_model(regressor, "model")


Elasticnet model (normalize=1.000000, l1_copy_x=0.000000):
Mean squared error for SLR on validation data = 0.11852351781694762
Mean absolute error for SLR on validation data = 0.2527641257902306
Mean R2 score for xgb on validation data = 0.9891693202503837


In [17]:
from sklearn.ensemble import RandomForestRegressor

In [18]:
with mlflow.start_run():
    n_estimators = 12
    rfr=RandomForestRegressor(n_estimators=n_estimators)
    rfr.fit(x_train,y_train)
    y_pred = rfr.predict(x_val)
    print("Mean squared error for RF on validation data =", mean_squared_error(y_val, y_pred))
    print("Mean absolute error for RF on validation data =", mean_absolute_error(y_val, y_pred))
    print("Mean R2 score for xgb on validation data =", r2_score(y_val, y_pred))
    mlflow.log_param("normalize", n_estimators)
    mlflow.log_metric("mse", mean_squared_error(y_val, y_pred))
    mlflow.log_metric("r2", r2_score(y_val, y_pred))
    mlflow.log_metric("mae", mean_absolute_error(y_val, y_pred))

    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

    # Model registry does not work with file store
    if tracking_url_type_store != "file":

        mlflow.sklearn.log_model(rfr, "model", registered_model_name="RandomForestRegressor")
    else:
        mlflow.sklearn.log_model(rfr, "model")


Mean squared error for RF on validation data = 0.018471459419053347
Mean absolute error for RF on validation data = 0.07655209474693409
Mean R2 score for xgb on validation data = 0.9983120779305186


# LSTM deep learning Model

In [19]:
features2 = preprocess_data(deep_train, deep_test)
features2 = features2.drop(['Date'], axis = 1)
features2.head()

,DayOfWeek,Open,Promo,SchoolHoliday,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,...,Assortment_a,Assortment_b,Assortment_c,"PromoInterval_Feb,May,Aug,Nov","PromoInterval_Jan,Apr,Jul,Oct","PromoInterval_Mar,Jun,Sept,Dec",StateHoliday_0,StateHoliday_a,StateHoliday_b,StateHoliday_c
0,0.25,0.0,1.0,1.0,-0.172078,0.666667,0.0,-1.0,0.0,0.0,...,1,0,0,0,1,0,1,0,0,0
1,0.00,0.0,1.0,1.0,-0.172078,0.666667,0.0,-1.0,0.0,0.0,...,1,0,0,0,1,0,1,0,0,0
2,-0.25,0.0,1.0,1.0,-0.172078,0.666667,0.0,-1.0,0.0,0.0,...,1,0,0,0,1,0,1,0,0,0
3,-0.50,0.0,1.0,1.0,-0.172078,0.666667,0.0,-1.0,0.0,0.0,...,1,0,0,0,1,0,1,0,0,0
4,-0.75,0.0,1.0,1.0,-0.172078,0.666667,0.0,-1.0,0.0,0.0,...,1,0,0,0,1,0,1,0,0,0


In [20]:
# reconstruct train and test sets
def reconstruct_sets(features, time_window):
    global x_train, x_val, y_train, y_val
    # global train_set
    # original train and test sets
    x_train = features.iloc[:len(train_features), :]
    x_test = features.iloc[len(train_features):, :]
    y_train = train_target
    
    # Transforming dataset to time series problem
    scaler = RobustScaler()
    for i in range(1, time_window+1):
        x_train["Sales_" + str(i)] = y_train.Sales.shift(i)
        x_train.fillna(0.0, inplace=True)
        x_train["Sales_" + str(i)] = scaler.fit_transform(x_train["Sales_" + str(i)].values.reshape(-1, 1))
    x_train.fillna(0.0, inplace=True)
    shift_test = x_train.copy()
    # train_set = pd.concat([x_train, y_train], axis=1)

    # updated train and validation sets
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = .20, random_state = 0)


    return x_train, x_val, y_train, y_val, x_test

x_train, x_val, y_train, y_val, x_test = reconstruct_sets(features2, 6)
# log transformation on target variable
y_train = np.log1p(y_train['Sales'])
y_val = np.log1p(y_val['Sales'])
x_train.shape, x_val.shape, y_train.shape, y_val.shape, x_test.shape

((813767, 35), (203442, 35), (813767,), (203442,), (41088, 29))

In [21]:
x_train_reshaped = x_train.values.reshape((x_train.shape[0], 1, x_train.shape[1]))
x_val_reshaped = x_val.values.reshape((x_val.shape[0],1, x_val.shape[1]))
x_train_reshaped.shape, x_val_reshaped.shape

((813767, 1, 35), (203442, 1, 35))

In [22]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

model_lstm = Sequential()
model_lstm.add(LSTM(8, activation='relu', input_shape=(x_train_reshaped.shape[1], x_train_reshaped.shape[2])))
model_lstm.add(Dense(2))
model_lstm.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])
model_lstm.summary()

2021-07-31 17:59:02.872356: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-07-31 17:59:02.872556: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-07-31 17:59:12.259805: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-07-31 17:59:12.291704: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-07-31 17:59:12.291820: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-07-31 17:59:12.291884: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 8)                 1408      
_________________________________________________________________
dense (Dense)                (None, 2)                 18        
Total params: 1,426
Trainable params: 1,426
Non-trainable params: 0
_________________________________________________________________


In [23]:
Denses = 2

In [24]:
with mlflow.start_run():
    history = model_lstm.fit(x_train_reshaped, y_train, validation_data=(x_val_reshaped, y_val),epochs=2, batch_size=12, verbose=2, shuffle=False)
    train_loss=history.history['loss'][-1]
    val_loss=history.history['val_loss'][-1]
    mse = history.history['mse'][-1]
    mlflow.log_param("network dense", Denses)
    mlflow.log_metric("loss", train_loss)
    mlflow.log_metric("val_loss", val_loss)
    mlflow.log_metric("mse", mse)

    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

    if tracking_url_type_store != "file":

        mlflow.keras.log_model(model_lstm, "LSTM model", registered_model_name="Sequential")
    else:
        mlflow.keras.log_model(model_lstm, "LSTM model")


2021-07-31 17:59:17.286737: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 113927380 exceeds 10% of free system memory.
2021-07-31 17:59:17.490775: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-07-31 17:59:17.592046: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1999965000 Hz


Epoch 1/2
67814/67814 - 62s - loss: 0.2751 - mse: 0.2751 - val_loss: 0.0362 - val_mse: 0.0362
Epoch 2/2
67814/67814 - 61s - loss: 0.0380 - mse: 0.0380 - val_loss: 0.0320 - val_mse: 0.0320


2021-07-31 18:01:21.234625: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [25]:
!mlflow ui

[2021-07-31 18:02:00 +0300] [5986] [INFO] Starting gunicorn 20.1.0
[2021-07-31 18:02:00 +0300] [5986] [INFO] Listening at: http://127.0.0.1:5000 (5986)
[2021-07-31 18:02:00 +0300] [5986] [INFO] Using worker: sync
[2021-07-31 18:02:00 +0300] [5989] [INFO] Booting worker with pid: 5989
^C
[2021-07-31 18:49:01 +0300] [5986] [INFO] Handling signal: int
[2021-07-31 18:49:01 +0300] [5989] [INFO] Worker exiting (pid: 5989)
